In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3

In [ ]:
# Load and preprocess the data
data_path = '/content/sample_data/brain tumour'
classes = ['no', 'yes']
img_size = 224

X = []
Y = []
for c in classes:
    path = os.path.join(data_path, c)
    class_num = classes.index(c)
    for img in os.listdir(path):
        img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        img_arr = cv2.resize(img_arr, (img_size, img_size))
        X.append(img_arr)
        Y.append(class_num)

X = np.array(X)
Y = np.array(Y)
Y = to_categorical(Y, num_classes=len(classes))

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/brain tumour/no'

In [ ]:
# Split the data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
# Load the pre-trained VGG16 and InceptionV3 models
base_model1 = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model2 = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))


87910968/87910968 [==============================] - 2s 0us/step


In [ ]:
# Create the hybrid model
x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.5)(x1)
predictions1 = Dense(len(classes), activation='softmax')(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.5)(x2)
predictions2 = Dense(len(classes), activation='softmax')(x2)

model = Model(inputs=[base_model1.input, base_model2.input], outputs=[predictions1, predictions2])


In [ ]:
# Freeze the layers in the pre-trained models
for layer in base_model1.layers:
    layer.trainable = False

for layer in base_model2.layers:
    layer.trainable = False


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 30

history = model.fit([X_train, X_train], [Y_train, Y_train], batch_size=batch_size, epochs=epochs, verbose=1, validation_data=([X_val, X_val], [Y_val, Y_val]))

Epoch 1/30
6/6 [==============================] - 195s 32s/step - loss: 44.1242 - dense_1_loss: 4.2369 - dense_3_loss: 39.8873 - dense_1_accuracy: 0.5969 - dense_3_accuracy: 0.5445 - val_loss: 8.1465 - val_dense_1_loss: 1.9716 - val_dense_3_loss: 6.1749 - val_dense_1_accuracy: 0.6042 - val_dense_3_accuracy: 0.8542
Epoch 2/30
6/6 [==============================] - 179s 31s/step - loss: 18.6599 - dense_1_loss: 2.1197 - dense_3_loss: 16.5402 - dense_1_accuracy: 0.7435 - dense_3_accuracy: 0.7225 - val_loss: 12.1771 - val_dense_1_loss: 1.4687 - val_dense_3_loss: 10.7083 - val_dense_1_accuracy: 0.7917 - val_dense_3_accuracy: 0.7083
Epoch 3/30
6/6 [==============================] - 178s 31s/step - loss: 15.6313 - dense_1_loss: 1.4610 - dense_3_loss: 14.1703 - dense_1_accuracy: 0.7801 - dense_3_accuracy: 0.6911 - val_loss: 14.1154 - val_dense_1_loss: 0.6832 - val_dense_3_loss: 13.4321 - val_dense_1_accuracy: 0.8542 - val_dense_3_accuracy: 0.7917
Epoch 4/30
6/6 [==============================] 

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True)


In [ ]:
score = model.evaluate([X_val, X_val], [Y_val, Y_val], verbose=0)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])



Validation loss: 1.3017311096191406
Validation accuracy: 0.702994167804718


In [ ]:
# Save the model
model.save('brain_tumor_detection.h5')
